In [1]:
!nvidia-smi

Mon Jun 19 10:10:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    45W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
##INSTALLING DEPENDENCIES

!pip install -Uqqq pip --progress-bar off
!pip install -qqq bitsandbytes==0.39.0
!pip install -qqq torch--2.0.1 --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/transformers.git@e03a9cc --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/peft.git@42a184f --progress-bar off
!pip install -qqq -U git+https://github.com/huggingface/accelerate.git@c9fbb71 --progress-bar off
!pip install -qqq datasets==2.12.0 --progress-bar off
!pip install -qqq loralib==0.1.1 --progress-bar off
!pip install einops

import os
import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 19.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement torch--2.0.1 (from versions: none)
ERROR: No matching distribution found for torch--2.0.1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.8 MB/s eta 0:00:00

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this infor

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('8013'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

In [5]:
#HUGGINGFACE LOGIN
notebook_login()

In [3]:
#LOAD APADTERS FROM HUGGINGFACE HUB
from peft import *
#change peft_model_id
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    load_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

peft_model_id = "nisaar/falcon7b-Indian_Law_150Prompts"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token


model = PeftModel.from_pretrained(model, peft_model_id)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
#GENERATION CONFIG
generation_config = model.generation_config
generation_config.max_new_tokens = 200
#temperature(range-->0.4-0.7)
generation_config_temperature = 0.3
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config_eod_token_id = tokenizer.eos_token_id
DEVICE = "cuda:0"

In [5]:
#GENERATE RESPONSE FUNCTION

def generate_response(question: str) -> str:
    prompt = f"""
    <human>: {question}
    <assistant>:
    """.strip()
    encoding = tokenizer(prompt, return_tensors="pt").to(DEVICE)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0],skip_special_tokens=True)

    assistant_start = '<assistant>:'
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start):].strip()

In [6]:
#GENERATE_RESPONE

prompt="Debate the merits and demerits of introducing simultaneous elections in India?"
print(generate_response(prompt))

Introducing simultaneous elections could increase voter participation, promote transparency, and address issues with political parties. However, it could also lead to political instability, conflicts between candidates, and logistical challenges. The debate would depend on factors like current electoral system, political parties, and public opinion on the merits.The debate would also touch on the costs and logistics of implementing simultaneous elections.The debate would also touch on the merits of a simultaneous election, which could increase voter participation, promote transparency, and address political party conflicts. However, it could also lead to political instability, as it might lead to a short-term focus on the incumbent government and could lead to conflicts between candidates.The debate would depend on the context and objectives of introducing simultaneous elections in India.The debate would touch on the merits and demerits of introducing simultaneous elections in the cont

In [7]:
#IMPORTING PANDAS

import pandas as pd
filePath="/content/testDataset.csv"
dataframe=pd.read_csv(filePath)
print(dataframe)

                                              Question  \
0                                                  NaN   
1                          General Knowledge Questions   
2    Debate the merits and demerits of introducing ...   
3    What are the duties of the President of India ...   
4    Write a legal memo on the issue of manual scav...   
..                                                 ...   
105  Describe the procedures for filing an appeal t...   
106  Describe the rights of an arrested person in I...   
107  Write an opinion piece on the recent changes t...   
108  Create an argument against the given legal sta...   
109  Describe how this Supreme Court judgement has ...   

                                        Not Fine Tuned  Fined Tuned  
0                                                  NaN          NaN  
1                                                  NaN          NaN  
2                                                  NaN          NaN  
3                      

In [11]:
#LOADING THE BENCHMARK_TEST_DATASET USING PANDAS

for i in range(1,len(dataframe)):
    if dataframe.loc[i, "Question"] in ["NaN", "General Knowledge Questions", "Fictional", "From the Training Data", "Data no shown to the model"]:
        continue
    else:
        value = dataframe.loc[i, "Question"]
        response = generate_response(value)
        print("--",response)
        dataframe.loc[i, 'Fined Tuned'] = response


-- Introducing simultaneous elections could increase voter participation, promote transparency, and address issues with political parties. However, it could also lead to political instability, conflicts between candidates, and logistical challenges. The debate would depend on factors like current electoral system, political parties, and public opinion on the merits.The debate would also touch on the costs and logistics of implementing simultaneous elections.The debate would also touch on the merits of a simultaneous election, which could increase voter participation, promote transparency, and address political party conflicts. However, it could also lead to political instability, as it might lead to a short-term focus on the incumbent government and could lead to conflicts between candidates.The debate would depend on the context and objectives of introducing simultaneous elections in India.The debate would touch on the merits and demerits of introducing simultaneous elections in the c

In [ ]:
dataframe.head()

In [11]:
dataframe.to_csv('output.csv', index=False)

# New section

In [ ]:
#CALLING FUNCTION

prompt = "Debate the merits and demerits of introducing simultaneous elections in India?"
print(generate_response(prompt))